# Probabilistic State Estimation

> This tutorial is implementation for [MIT state estimation class](https://ocw.mit.edu/courses/6-01sc-introduction-to-electrical-engineering-and-computer-science-i-spring-2011/pages/unit-4-probability-and-planning/state-estimation/)

In [ ]:
#| default_exp probabilistic_state_estimation

In [ ]:
#| hide
import random

In [ ]:
#| export
class DDist:
    def __init__(self, dictionary:dict) -> None:
        self.d = dictionary
    def prob(self, elt # an element of the domain of this distribution
             ) -> float:
        if elt in self.d:
            return self.d[elt]
        else:
            return 0
    def support(self):
        return [k for k in self.d.keys() if self.prob(k) > 0]
    @property
    def dist(self):
        return self.d
    def draw(self):
        r = random.random()
        sum = 0.0
        for val in self.support():
            sum += self.prob(val)
            if r < sum:
                return val

## Conditional Probabilities

In [ ]:
def TgivenD(D):
    if D == 'disease':
        return DDist({'positive' : 0.99, 'negative' : 0.01})
    elif D == 'nodisease':
        return DDist({'positive' : 0.001, 'negative' : 0.999})
    else:
        raise Exception('invalid value for D')

In [ ]:
TgivenD('disease').prob('negative')

0.01

## Joint Probability Distribution

> Excercise to estimate the joint distribution probability given $P(A)$ and $P(B|A)$.

![General definition for join probability distribution for an arbitrary number of random variables.](joint_prob_dist.png)

In [ ]:
#| hide
# P(B | A)
def PBgA(a:DDist) -> DDist:
    if a == 'a1':
        return DDist({'b1' : 0.7, 'b2' : 0.3})
    else:
        return DDist({'b1' : 0.2, 'b2' : 0.8})
# P(A)
PA = DDist({'a1' : 0.9, 'a2' : 0.1})
print('PA.prob({}): {}'.format('a1', PA.prob('a1')))

# P(B | A = a1)
print('PBgA({}).prob({}): {}'.format('a1', 'b1', PBgA('a1').prob('b1')))

PA.prob(a1): 0.9
PBgA(a1).prob(b1): 0.7


In [ ]:
#| export
# Calculate the join distribution probabilities.
class JDist(DDist):
    def __init__(self, PA:DDist # P(A)
                 ,PBgA:callable # condtional probability P(B|A)
                 ) -> DDist:
        PAB = dict()
        for a in PA.dist.keys():
            PBgA_a = PBgA(a)
            for b in PBgA_a.dist.keys():
                PAB[(a,b)] = PA.prob(a) * PBgA_a.prob(b)
        self.d = PAB
    def marginalizeOut(self, 
                       idx:int # which random variable ..?
                       ) -> DDist:
        # A = a
        symbolKeys = self.d.keys()
        for key in symbolKeys:
            s = key[idx]
            
        return
    

In [ ]:
JDist(PA, PBgA).dist

{('a1', 'b1'): 0.63,
 ('a1', 'b2'): 0.27,
 ('a2', 'b1'): 0.020000000000000004,
 ('a2', 'b2'): 0.08000000000000002}

## Marginalization
> Marginalize out A or B

In [ ]:
#| hide
from nbdev import show_doc

In [ ]:
show_doc(DDist.prob)

---

### DDist.prob

>      DDist.prob (elt)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| elt |  | an element of the domain of this distribution |
| **Returns** | **float** |  |

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()